### Imports

In [ ]:
from SPDC_Simulation import SPDC_Simulation
from Crystal import Crystal
from Laser import Laser
from Materials import KTP1, KTP2, KTP3

### Run the simulation

In [ ]:
# Initialize crystal parameters
Lc = 46.175e-6                # Coherence length (in meters)
Lo = 30e-3                    # Crystal length (in meters)
T = 22.15                     # Temperature (in degrees Celsius)
w = 18e-6                     # Domain width parameter (in meters)

material = KTP1()
crystal = Crystal(Lc, Lo, T, w, material, spdc="type-II")

# Initialize laser parameters
lambda_2w = 775e-9            # Central wavelength of the pump (in meters)
pulse_duration = 1.7e-12      # Pulse duration (in seconds)
laser = Laser(lambda_2w, pulse_duration)

# Generate the poling pattern in the crystal
crystal.generate_poling(laser=laser, mode='periodic', resolution=5)

# Create and run the simulation
simulation = SPDC_Simulation(crystal, laser)

simulation.run_simulation(steps=100, dev=5)
print(f'The simulated purity is P={simulation.Purity}')
print(f'The simulated Schmidt number K={simulation.K}')

### Plot the results

In [ ]:
simulation.plot_jsa()

In [ ]:
simulation.plot_schmidt_coefficients()